# Data Access

In [1]:
import os
import gzip
import json
import isbnlib
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime

%load_ext autoreload
%autoreload 2

In [2]:
data_base_path = "../data/"
base_path = "/mnt/data/public/openlibrary/20221007/"
os.listdir(base_path)

['robots.txt.tmp',
 'ol_dump_editions_latest.txt.gz',
 'ol_dump_works_latest.txt.gz',
 'ol_dump_authors_latest.txt.gz',
 'ol_dump_latest.txt.gz',
 'ol_cdump_latest.txt.gz',
 'ol_dump_ratings_latest.txt.gz',
 'ol_dump_reading-log_latest.txt.gz']

In [3]:
print(os.getcwd())

/home/msds2025/acastillo/dmw2-2025/lab1/dmw2/notebooks


In [5]:
files = {
    "ol_dump_works_latest.txt.gz": {
        "total_lines": 26201547,
        "max_last_modified_date": datetime(2022, 9, 30, 23, 59, 50, 674192),
    },
    "ol_dump_editions_latest.txt.gz": {
        "total_lines": 36657084,
        "max_last_modified_date": datetime(2022, 9, 30, 23, 59, 50, 674192),
    },
    "ol_dump_authors_latest.txt.gz": {
        "total_lines": 9867148,
        "max_last_modified_date": datetime(2022, 9, 30, 23, 59, 50, 674192),
    },
    "ol_dump_reading-log_latest.txt.gz": {
        "total_lines": 4355023,
        "max_last_modified_date": None,
    },
    "editions_after_2015.txt": {
        "total_lines": 4259552,
    },
    "filtered_works_with_editions.txt": {
        "total_lines": 909883
    },
    "filtered_authors_with_works_and_editions.txt": {
        "total_lines": 853086,
    }
}

In [6]:
files

{'ol_dump_works_latest.txt.gz': {'total_lines': 26201547,
  'max_last_modified_date': datetime.datetime(2022, 9, 30, 23, 59, 50, 674192)},
 'ol_dump_editions_latest.txt.gz': {'total_lines': 36657084,
  'max_last_modified_date': datetime.datetime(2022, 9, 30, 23, 59, 50, 674192)},
 'ol_dump_authors_latest.txt.gz': {'total_lines': 9867148,
  'max_last_modified_date': datetime.datetime(2022, 9, 30, 23, 59, 50, 674192)},
 'ol_dump_reading-log_latest.txt.gz': {'total_lines': 4355023,
  'max_last_modified_date': None},
 'editions_after_2015.txt': {'total_lines': 4259552},
 'filtered_works_with_editions.txt': {'total_lines': 909883},
 'filtered_authors_with_works_and_editions.txt': {'total_lines': 853086}}

### Editions
Extract only editions published after 2015

In [7]:
# !zgrep -E '"publish_date": "(20(1[6-9]|[2-9][0-9]))([^"]*)?"' /mnt/data/public/openlibrary/20221007/ol_dump_editions_latest.txt.gz > /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/editions_after_2015.txt

In [8]:
!wc -l /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/editions_after_2015.txt

4259552 /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/editions_after_2015.txt


In [9]:
file_name = "editions_after_2015.txt"
total_lines = files["editions_after_2015.txt"]["total_lines"]
total_lines

4259552

In [10]:
df_dict = {}
with open(os.path.join(data_base_path, file_name), "r", encoding="utf-8") as f:
    for i, line in enumerate(tqdm(f, total=total_lines, desc="Processing lines")):
        # if i >= 200:
        #     break
        
        parts = line.strip().split("\t", maxsplit=4)
        if len(parts) < 5:
            skipped_lines += 1
            continue
        
        record_type, id, revision, timestamp, json_data = parts

        try:
            data_dict = json.loads(json_data)


            try:
                works_id = data_dict.get("works", np.nan)[0]["key"]
            except TypeError:
                continue

            try:
                description = data_dict.get("description", np.nan).get("value", np.nan)
            except AttributeError:
                description = data_dict.get("description", np.nan)
                
            row_data = {
                "record_type": record_type,
                "book_id": id,
                "publishers": data_dict.get("publishers", np.nan),
                "title": data_dict.get("title", np.nan),
                "description": description,
                "first_sentence": data_dict.get("first_sentence", np.nan),
                "subjects": data_dict.get("subjects", np.nan),
                "date_created": data_dict.get("created", np.nan).get("value", np.nan),
                "isbn_13": data_dict.get("isbn_13", "000"),
                "publish_date": data_dict.get("publish_date", np.nan),
                "publish_country": data_dict.get("publish_country", np.nan),
                "number_of_pages": data_dict.get("number_of_pages", np.nan),
                "latest_revision": data_dict.get("latest_revision", np.nan),
                "work_id": works_id
            }

            df_dict[i] = row_data

            # if all(value is not np.nan for value in row_data.values()):
            #     df_dict[i] = row_data
    
        except json.JSONDecodeError:
            print(f"JSON Decode Error on line {i}")

Processing lines: 100%|██████████| 4259552/4259552 [01:29<00:00, 47539.70it/s]


In [11]:
df = pd.DataFrame.from_dict(df_dict, orient="index")
df.head()

,record_type,book_id,publishers,title,description,first_sentence,subjects,date_created,isbn_13,publish_date,publish_country,number_of_pages,latest_revision,work_id
0,/type/edition,/books/OL24367788M,[PUBLICAT S. A.],Akademia Canterwood: Wysokie Aspiracje,W elitarnym gronie\r\n\r\nPo powrocie z ferii ...,NaN,NaN,2010-09-28T19:36:49.368985,[9788327155238],2016,NaN,213.0,5,/works/OL15386091W
1,/type/edition,/books/OL25896697M,[Barbour],Too Blessed to Be Stressed,NaN,NaN,NaN,2016-03-16T12:15:21.831726,[9781634095693],2016,NaN,NaN,4,/works/OL17321171W
2,/type/edition,/books/OL25896725M,[Montena],"Fuera llueve, dentro también, ¿paso a buscarte?",NaN,NaN,NaN,2016-03-16T16:42:51.036657,[9788490435656],2016,NaN,NaN,3,/works/OL17321197W
3,/type/edition,/books/OL25898027M,[Pálido Fuego],Satin Island,NaN,NaN,NaN,2016-03-23T09:01:37.048834,[9788494365577],2016,NaN,NaN,3,/works/OL17322236W
4,/type/edition,/books/OL25898429M,[Pregunta Ediciones],La península de Cilemaga,NaN,NaN,NaN,2016-03-28T07:53:55.863861,[9788494519512],2016,NaN,NaN,4,/works/OL17322587W


In [12]:
reduced_df = df.dropna(subset=["subjects", "title"])

all_lines = list(reduced_df["work_id"])
with open(os.path.join(data_base_path, "work_ids.txt"), "w") as f:
    for line in all_lines:
        f.write(f"{line.strip()}\n")

### Works

In [17]:
# !zgrep -F -f data/work_ids.txt /mnt/data/public/openlibrary/20221007/ol_dump_works_latest.txt.gz > /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/filtered_works_with_editions.txt

In [13]:
!wc -l /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/filtered_works_with_editions.txt

909883 /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/filtered_works_with_editions.txt


In [14]:
file_name = "filtered_works_with_editions.txt"
total_lines = files["filtered_works_with_editions.txt"]["total_lines"]
total_lines

909883

In [15]:
df_dict = {}

with open(os.path.join(data_base_path, file_name), "r", encoding="utf-8") as f:
    for i, line in enumerate(tqdm(f, total=total_lines, desc="Processing lines")):
        # if i >= 100:
        #     break
        
        parts = line.strip().split("\t", maxsplit=4)
        if len(parts) < 5:
            skipped_lines += 1
            continue
        
        record_type, id, revision, timestamp, json_data = parts

        try:
            data_dict = json.loads(json_data)

            try:
                author_id = data_dict.get("authors", [{}])[0].get("author", {}).get("key", np.nan)
            except (IndexError, KeyError, AttributeError):
                author_id = data_dict["authors"][0]["author"]

            row_data = {
                "record_type": record_type,
                "work_id": id,
                "title": data_dict.get("title", np.nan),
                "subjects": data_dict.get("subjects", np.nan),
                "last_modified": data_dict.get("created", {}).get("value", np.nan),
                "latest_revision": data_dict.get("latest_revision", np.nan),
                "revision": data_dict.get("revision", np.nan),
                "author_type": data_dict.get("authors", [{}])[0].get("type", np.nan),
                "author_id": author_id,
            }

            df_dict[i] = row_data

            # if all(value is not np.nan for value in row_data.values()):
            #     df_dict[i] = row_data
    
        except json.JSONDecodeError:
            print(f"JSON Decode Error on line {i}")

Processing lines: 100%|██████████| 909883/909883 [00:10<00:00, 88389.53it/s]


In [16]:
df = pd.DataFrame.from_dict(df_dict, orient="index")
df.head()

,record_type,work_id,title,subjects,last_modified,latest_revision,revision,author_type,author_id
0,/type/work,/works/OL10613658W,The unseen voice,"[Radio broadcasting, australia]",2009-12-11T03:16:38.229415,5,5,{'key': '/type/author_role'},/authors/OL4416271A
1,/type/work,/works/OL10677836W,"The later Middle Ages in England, 1216-1485","[13th century, 14th century, History, Lancaste...",2009-12-11T03:24:56.987006,4,4,{'key': '/type/author_role'},/authors/OL4455359A
2,/type/work,/works/OL11188173W,The nature of Christian worship,[Worship],2009-12-11T04:11:24.980995,3,3,{'key': '/type/author_role'},/authors/OL4674047A
3,/type/work,/works/OL11286673W,Heating and cooling of buildings,"[Air conditioning, Design and construction, Eq...",2009-12-11T04:23:11.323166,4,4,{'key': '/type/author_role'},/authors/OL4734998A
4,/type/work,/works/OL11821146W,Dynamische Fiskalpolitik,"[Konjunkturmodell, Realer-Konjunkturzyklus-The...",2009-12-11T05:27:21.875721,4,4,{'key': '/type/author_role'},/authors/OL5045591A


In [18]:
cleaned_works = df.dropna(subset=["author_type"])
cleaned_works

all_lines = list(cleaned_works["author_id"].str.strip())
with open(os.path.join(data_base_path, "author_ids.txt"), "w", encoding="utf-8") as f:
    for line in all_lines:
        f.write(f"{line.strip()}\n")

### Authors

In [31]:
# !split -l 10000 data/author_ids.txt /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/split/authors_split_

# !ls data/split/authors_split_* | xargs -I{} zgrep -F -f {} /mnt/data/public/openlibrary/20221007/ol_dump_authors_latest.txt.gz >> /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/filtered_authors_with_works_and_editions.txt


In [19]:
!wc -l /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/filtered_authors_with_works_and_editions.txt

853086 /home/msds2025/acastillo/dmw2-2025/lab1/dmw2/data/filtered_authors_with_works_and_editions.txt


In [20]:
file_name = "filtered_authors_with_works_and_editions.txt"
total_lines = files["filtered_authors_with_works_and_editions.txt"]["total_lines"]
total_lines

853086

In [21]:
df_dict = {}

with open(os.path.join(data_base_path, file_name), "r", encoding="utf-8") as f:
    for i, line in enumerate(tqdm(f, total=total_lines, desc="Processing lines")):
        # if i >= 100:
        #     break
        
        parts = line.strip().split("\t", maxsplit=4)
        if len(parts) < 5:
            skipped_lines += 1
            continue

        record_type, id, revision, timestamp, json_data = parts

        try:
            data_dict = json.loads(json_data)

            row_data = {
                "record_type": record_type,
                "author_id": id,
                "birth_date": data_dict.get("birth_date", np.nan),
                "death_date": data_dict.get("death_date", np.nan),
                "location": data_dict.get("location", np.nan)
            }
            
            df_dict[i] = row_data

            # if all(value is not np.nan for value in row_data.values()):
            #     df_dict[i] = row_data
    
        except json.JSONDecodeError:
            print(f"JSON Decode Error on line {i}")
        

Processing lines: 100%|██████████| 853086/853086 [00:05<00:00, 146740.39it/s]


In [22]:
df = pd.DataFrame.from_dict(df_dict, orient="index")
df.head()

,record_type,author_id,birth_date,death_date,location
0,/type/author,/authors/OL10259922A,NaN,NaN,NaN
1,/type/author,/authors/OL1521743A,NaN,NaN,NaN
2,/type/author,/authors/OL19377A,NaN,NaN,NaN
3,/type/author,/authors/OL2691435A,NaN,NaN,NaN
4,/type/author,/authors/OL3145407A,NaN,NaN,NaN
